In [16]:
import cv2
import zmq
import time
import torch
import pickle
import numpy as np
import gymnasium as gym
from process_frame import process_frame
from stable_baselines3 import PPO

In [ ]:
class CarlaEnv(gym.Env):
    def __init__(self, target_gnss, timeout_sec=60):
        super(CarlaEnv, self).__init__()
        
        context = zmq.Context()
        self.socket = context.socket(zmq.REQ)
        self.socket.connect("tcp://localhost:6501")
        
        self.action_space = gym.spaces.Box(low=np.array([-1.0, 0.0]), high=np.array([1.0, 1.0]), dtype=np.float32)
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(1, 84, 84), dtype=np.uint8)
        
        self.prev_info = None
        self.target_gnss = target_gnss
        self.timeout_sec = timeout_sec
        self.reset_time = time.time() 
    
    def step(self, action):
        data = {"action": action.tolist()}
        self.socket.send(pickle.dumps(data))
        
        response = pickle.loads(self.socket.recv())
        obs = self._process_obs(response["image"])

        reward, done = self._calculate_reward(response)
        info = response

        self.prev_info = response
        return obs, reward, done, False, info
    
    def reset(self, seed=None, options=None):
        # === Reset Carla ===
        self.socket.send(pickle.dumps({"command": "reset"}))
        _ = self.socket.recv()

        # === Wait for obs ===
        self.socket.send(pickle.dumps({"action": [0.0, 0.0]}))
        response = pickle.loads(self.socket.recv())

        obs = self._process_obs(response["image"])
        self.prev_info = response
        self.reset_time = time.time()
        return obs, {}

    def _process_obs(self, image_bytes):
        img = np.frombuffer(image_bytes, dtype=np.uint8).reshape((84, 84))  # (H, W)
        return np.expand_dims(img, axis=0)  # (1, H, W)

    def _calculate_reward(self, info):
        reward = 0
        done = False

        # === Collision ===
        if info["collision"] is not None:
            return -100, True

        # === Lane Invasion ===
        if info["lane_invaded"]["violated"]:
            return -50, True

        # === Target Reached ===
        distance = gnss_distance(info["gnss"], self.target_gnss)
        if distance < 2.0:
            return 100, True

        # === Too slow / stuck ===
        if is_stuck(info, self.prev_info):
            reward -= 10

        # === Time penalty ===
        if time.time() - self.reset_time > self.timeout_sec:
            return -50, True

        # === Progress reward ===
        prev_dist = gnss_distance(self.prev_info["gnss"], self.target_gnss) if self.prev_info else distance + 1
        reward += (prev_dist - distance) * 2

        return reward, done

    def render(self):
        pass

    def close(self):
        self.socket.close()
        

In [31]:
def send_action(steer: float, throttle: float):
    action = {
        "action": [steer, throttle]
    }
    socket.send(pickle.dumps(action))  # Serialize & send
    response = socket.recv()           # Receive serialized reply
    data = pickle.loads(response)      # Deserialize

    return data

# ==== Decode image from bytes ====
def decode_image(image_bytes):
    np_arr = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
    return img

# ==== Main loop ====
for step in range(200):
    steer = 0.1
    throttle = 1
    obs = send_action(steer, throttle)

    # Extract observation
    # try:
        # image = decode_image(obs["image"])
    gnss = obs["gnss"]
    collision = obs["collision"]
    imu = obs["imu"]
    timestamp = obs["timestamp"]
    lane_invaded = obs["lane_invaded"]
    
    # procsses_image = process_frame(image)
    
    print(f"[Step {step}] GNSS: {gnss} | Time: {timestamp} | Collision: {collision} | imu: {imu} | lane: {lane_invaded}")
    # print(f"lane: {lane_invaded}")
    if lane_invaded['violated'] == True:
        print(lane_invaded["last_event"])
        print(lane_invaded)
        # cv2.imshow("Invasion", image)

    # Optional: show image
    # cv2.imshow("Camera", procsses_image)
    # cv2.waitKey(1)

    # except Exception as e:
    #     print(lane_invaded["last_event"])
    #     print(lane_invaded)
    #     print(e)
    #     # print(f"[ERROR] Failed to process observation: {e}")
    #     # print(f"[DEBUG] Image shape before processing: {image.shape}")
    #     pass
    #     # break

    time.sleep(0.05)  # slow it down for now
cv2.destroyAllWindows()

# Reset Carla at end (optional)
# reset_command = {"command": "reset"}
# socket.send(pickle.dumps(reset_command))
# _ = socket.recv()

[Step 0] GNSS: {'latitude': -0.0006907111211233996, 'longitude': -0.0005086639018868944} | Time: 1744096889.5446916 | Collision: {'frame': 1010892, 'intensity': 75.92672729492188, 'other_actor': 'static.building'} | imu: {'orientation': {'w': 0.9715170915447163, 'x': 0.0, 'y': 0.0, 'z': 0.23696949347225096}, 'orientation_covariance': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'angular_velocity': {'x': -0.005229749251157045, 'y': -0.0006241102237254381, 'z': 0.001992671051993966}, 'angular_velocity_covariance': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'linear_acceleration': {'x': 0.7489782571792603, 'y': -0.30708980560302734, 'z': 9.789708137512207}, 'linear_acceleration_covariance': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]} | lane: {'violated': False, 'last_event': ['<carla.libcarla.LaneMarking object at 0x741cf56079f0>']}
[Step 1] GNSS: {'latitude': -0.0006907140681704504, 'longitude': -0.0005086638333508771} | Time: 1744096889.6308887 | Collision: {'frame': 1010895, 'i